In [2]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import pprint
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from math import sqrt
from sklearn import ensemble
from sklearn.inspection import permutation_importance

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.metrics import r2_score
import xgboost as xgb
%matplotlib inline


from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
pd.set_option('display.max_rows', 20)
pd.set_option('display.float_format', '{:.2f}'.format)
np.set_printoptions(suppress=True)

In [5]:
df = pd.read_csv("/content/sample_data/WA_Cngstn.csv")
df.head()


,Type,Severity,TMC,StartTime(UTC),EndTime(UTC),LocationLat,LocationLng,Distance(mi),Side,ZipCode,IsAnomaly,Duration
0,3,1,74,2016-11-30 23:35:00,2016-12-01 00:55:09,47.01,-122.91,1.49,-1,98501.00,0,80
1,3,2,73,2016-12-01 00:52:00,2016-12-01 01:04:04,47.61,-122.33,1.49,-1,98104.00,0,12
2,3,1,74,2016-12-01 01:08:00,2016-12-01 01:26:07,47.96,-122.20,2.30,-1,98203.00,0,18
3,3,1,74,2016-12-01 01:02:00,2016-12-01 01:24:00,47.56,-122.19,2.59,-1,98006.00,0,22
4,3,1,74,2016-12-01 01:19:00,2016-12-01 01:31:07,47.62,-122.33,1.64,-1,98101.00,0,12


In [6]:
df_prune = df.drop(['Type', 'TMC', 'Distance(mi)', 'EndTime(UTC)', 'Side', 'IsAnomaly', 'ZipCode'], axis = 1)
def Mon(x):
    return x.split(" ")[0].split('-')[1]
def Day(x):
    return x.split(" ")[0].split('-')[2]
def Hour(x):
    return x.split(" ")[1].split(':')[0]
def Minute(x):
    return x.split(" ")[1].split(':')[1]
df_prune['StartMonth'] = df_prune['StartTime(UTC)'].apply(lambda x: Mon(x)).astype(int)
df_prune['StartDay'] = df_prune['StartTime(UTC)'].apply(lambda x: Day(x)).astype(int)
df_prune['StartHour'] = df_prune['StartTime(UTC)'].apply(lambda x: Hour(x)).astype(int)
df_prune['StartMinute'] = df_prune['StartTime(UTC)'].apply(lambda x: Minute(x)).astype(int)
df_prune.drop('StartTime(UTC)', inplace=True, axis = 1)
df_prune.isnull().sum().sum()
df_prune.dropna(inplace=True)
df_prune.head()

,Severity,LocationLat,LocationLng,Duration,StartMonth,StartDay,StartHour,StartMinute
0,1,47.01,-122.91,80,11,30,23,35
1,2,47.61,-122.33,12,12,1,0,52
2,1,47.96,-122.20,18,12,1,1,8
3,1,47.56,-122.19,22,12,1,1,2
4,1,47.62,-122.33,12,12,1,1,19


In [7]:
train, test = train_test_split(df_prune, test_size = 0.2)
target = train.Duration
train.drop(['Duration'],axis = 1 , inplace = True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [12]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               1024      
_________________________________________________________________
dense_7 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_8 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_9 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total params: 165,889
Trainable params: 165,889
Non-trainable params: 0
_________________________________________________________________


In [0]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [14]:
NN_model.fit(train, target, epochs=500, batch_size=1000, validation_split = 0.2, callbacks=callbacks_list)

Train on 357619 samples, validate on 89405 samples
Epoch 1/500
357619/357619 [==============================] - 10s 28us/step - loss: 20.4799 - mean_absolute_error: 20.4799 - val_loss: 19.8282 - val_mean_absolute_error: 19.8282

Epoch 00001: val_loss improved from inf to 19.82816, saving model to Weights-001--19.82816.hdf5
Epoch 2/500
357619/357619 [==============================] - 10s 27us/step - loss: 19.5897 - mean_absolute_error: 19.5897 - val_loss: 19.7597 - val_mean_absolute_error: 19.7597

Epoch 00002: val_loss improved from 19.82816 to 19.75970, saving model to Weights-002--19.75970.hdf5
Epoch 3/500
357619/357619 [==============================] - 10s 27us/step - loss: 19.5484 - mean_absolute_error: 19.5484 - val_loss: 19.7470 - val_mean_absolute_error: 19.7470

Epoch 00003: val_loss improved from 19.75970 to 19.74702, saving model to Weights-003--19.74702.hdf5
Epoch 4/500
357619/357619 [==============================] - 9s 27us/step - loss: 19.5184 - mean_absolute_error: 19.5

In [0]:
wights_file = 'Weights-488--18.80487.hdf5'
NN_model.load_weights(wights_file) # load it
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [0]:
predictions = NN_model.predict(test.drop('Duration', axis=1))

In [0]:
from sklearn.metrics import r2_score

In [19]:
r2_score(test.Duration, predictions)

0.0007101090759975559